In [1]:
import pandas as pd
import numpy as np
from haversine import haversine
import os

os.getcwd()

'/home/jian/Projects/Savers/TA_creation/CA_2_stores_2154_2158'

In [10]:
df_CA_zip_centers=pd.read_csv("/home/jian/Docs/Geo_mapping/Canadian_Postal_Codes/canada-postal-code-database-BUSINESS-CSV/canada-postal-code-database-BUSINESS.csv",
                              dtype=str,usecols=['PostalCode','Latitude','Longitude'])
df_CA_zip_centers['Latitude']=df_CA_zip_centers['Latitude'].astype(float)
df_CA_zip_centers['Longitude']=df_CA_zip_centers['Longitude'].astype(float)
print(df_CA_zip_centers.shape,df_CA_zip_centers['PostalCode'].nunique())

df_CA_zip_centers=df_CA_zip_centers.drop_duplicates()
print(df_CA_zip_centers.shape,df_CA_zip_centers['PostalCode'].nunique())


(1115041, 3) 873339
(873339, 3) 873339


In [15]:
df_CA_zip_centers['zip_center']=df_CA_zip_centers[['Latitude','Longitude']].values.tolist()
dict_CA_zip_centers=df_CA_zip_centers.set_index("PostalCode").to_dict()['zip_center']

In [19]:
dict_store_centers={}
dict_store_centers.update({"2154":[43.4557431,-79.6871351,'L6J2W8']})
dict_store_centers.update({"2158":[43.7295257,-79.2844664,'M1L 2M6']})

df_output=pd.DataFrame(index=dict_store_centers.keys()).reset_index().rename(columns={"index":"location_id"})
df_output['latitude']=df_output['location_id'].apply(lambda x: dict_store_centers[x][0])
df_output['longitude']=df_output['location_id'].apply(lambda x: dict_store_centers[x][1])
df_output['store_zip']=df_output['location_id'].apply(lambda x: dict_store_centers[x][0])


In [21]:
df_zip_store=pd.DataFrame()
for ind,row in df_output.iterrows():
    store_centers=(row['latitude'],row['longitude'])
    df_store=pd.DataFrame()
    for zip_cd,v in dict_CA_zip_centers.items():
        dist=haversine(store_centers,v,miles=True)
        if dist<=5:
            df=pd.DataFrame({"dist":dist},index=[zip_cd])
            df=df.reset_index().rename(columns={"index":"zip"})
            df_store=df_store.append(df)
    df_store['location_id']=row['location_id']
    df_zip_store=df_zip_store.append(df_store)
    
    

In [22]:
df_output=pd.merge(df_output,df_zip_store,on="location_id",how="left")

In [53]:
df_output.to_csv("./Savers_zip_6_2_stores_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [27]:
df_CC=pd.read_excel("./Savers_New2Stores_TA 20191017 CC.xlsx",dtype=str)

In [41]:
df_CC_zip_3=df_CC[['location_id','Zip first 3']].drop_duplicates()
df_CC_zip_3_included=df_CC.drop_duplicates()
df_CC_zip_3_included=df_CC_zip_3_included.groupby(['location_id','Zip first 3'])['zip'].nunique().to_frame().reset_index()
df_CC_zip_3_included=df_CC_zip_3_included.rename(columns={"zip":"zip_count_in_TA"})

In [46]:
df_zip_total=df_CA_zip_centers[['PostalCode']].drop_duplicates()
df_zip_total['Zip first 3']=df_zip_total['PostalCode'].apply(lambda x: x[:3])
df_zip_total=df_zip_total.groupby("Zip first 3")['PostalCode'].nunique().to_frame().reset_index()
df_zip_total=df_zip_total.rename(columns={"PostalCode":"zip_count_Total_of_zip3"})

In [47]:
df_check_pctg=pd.merge(df_CC_zip_3_included,df_zip_total,on="Zip first 3",how="left")

In [51]:
import datetime
writer=pd.ExcelWriter("./Savers_zip_6_to_3_QC_with_Pctg_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_check_pctg.to_excel(writer,"QC_pctg",index=False)
writer.save()